# Ejercicio 1

El área financiera requiere de un archivo de control que relacione las cuentas internas de la empresa con la descripción de las cuentas definidas por entidades regulatorias. Para ello, se cuenta con el archivo “Cuentas.xlsx”, el cual contiene 4 hojas:

* Cuentas internas: Lista de identificadores utilizados dentro de la empresa para las cuentas. Las cuentas tienen subcuentas, que pueden extenderse en diferentes niveles (más de 3).
* Cuentas regulatorias: Estas cuentas son las definidas por las entidades regulatorias
* Relación: Mantiene una relación de las cuentas regulatorias con cuentas internas (esta puede estar dada en cualquier nivel)
* Ejemplo de resultado

El objetivo del ejercicio es:
1. A partir de la hoja 1, encontrar el patrón con el que se identifican las subcuentas de una cuenta (o subcuenta) dada.
2. Organizar la información como en la hoja “Ejemplo de resultado” para todas las subcuentas dentro del esquema de todas las cuentas internas relacionadas a una cuenta regulatoria.


In [2]:
#importar librería pandas
import pandas as pd

In [47]:
#Cargas como dataframe cada una de las tablas del archivo
df1 = pd.read_excel('/content/Cuentas.xlsx',sheet_name='Cuentas internas')
df2 = pd.read_excel('/content/Cuentas.xlsx',sheet_name='Cuentas regulatorias')
df3 = pd.read_excel('/content/Cuentas.xlsx',sheet_name='Relación')
#Ver tabla 1
df1.head()

,CUENTA
0,1
1,11
2,1101
3,110101
4,110103


In [48]:
#ver tabla 2
df2.head()

,CUENTA,Descripción
0,1000,100000000000 ACTIVO
1,2000,100200001001 Efectivo y equivalentes de efectivo
2,3000,100200102001 Caja
3,4000,100200102002 Bancos
4,5000,100200203001 Depósitos en Banco de México


In [49]:
#tabla 3
df3.head()

,Cuenta regulatoria,Cuenta interna
0,3000,1101
1,5000,110201
2,6000,110202
3,7000,110203
4,9000,110303


In [50]:
#ver info de la tabla, tipo da datos, null, etc.
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1454 entries, 0 to 1453
Data columns (total 2 columns):
 #   Column              Non-Null Count  Dtype
---  ------              --------------  -----
 0   Cuenta regulatoria  1454 non-null   int64
 1   Cuenta interna      1454 non-null   int64
dtypes: int64(2)
memory usage: 22.8 KB


El patrón que logro observar es que las subcuentas de la cuenta puden indentificarse con las iniciales de la cuenta. Es decir, si la cuenta es 1101, la subcuentas empiezan con 1101******, indpendientemente de que la subcuenta sea de nivel uno, dos o tres, etc.


In [52]:
# Convertir las columnas a arrays para poder iterar en ciclo FOR
CeuntasInternas = df3['Cuenta interna']
arrayCI = CeuntasInternas.astype(str).to_numpy()  # Convertir a strings
print(arrayCI)

Totalcuentas = df1['CUENTA']
arrayC = Totalcuentas.astype(str).to_numpy()  # Convertir a strings
print(arrayC)

# Crear una lista para almacenar las cuentas y subcuentas clasificadas
data = []

# Clasificar las cuentas según las cuentas principales
for cuenta in arrayCI:
    for subcuenta in arrayC:
        # Verificar si la subcuenta comienza con la cuenta y no son iguales
        if subcuenta.startswith(cuenta) and cuenta != subcuenta:
            data.append({"Cuenta interna": cuenta, "Subcuentas": subcuenta})

# Crear un DataFrame con el resultado
df = pd.DataFrame(data)

# Mostrar el DataFrame
print(df)


['1101' '110201' '110202' ... '570102' '6601' '5601']
['1' '11' '1101' ... '960102' '96010201' '96010202']
     Cuenta interna    Subcuentas
0              1101        110101
1              1101        110103
2              1101        110104
3              1101        110105
4              1101        110106
...             ...           ...
4626         649190  649190040210
4627         649190  649190040211
4628         649190      64919005
4629         649190      64919008
4630         649190    6491900801

[4631 rows x 2 columns]


In [53]:
df.info()

#convertir la columna "cuenta interna" a núm para hacer el JOIN
df['Cuenta interna'] = df['Cuenta interna'].astype('int64')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4631 entries, 0 to 4630
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Cuenta interna  4631 non-null   object
 1   Subcuentas      4631 non-null   object
dtypes: object(2)
memory usage: 72.5+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4631 entries, 0 to 4630
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Cuenta interna  4631 non-null   int64 
 1   Subcuentas      4631 non-null   object
dtypes: int64(1), object(1)
memory usage: 72.5+ KB


In [54]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1454 entries, 0 to 1453
Data columns (total 2 columns):
 #   Column              Non-Null Count  Dtype
---  ------              --------------  -----
 0   Cuenta regulatoria  1454 non-null   int64
 1   Cuenta interna      1454 non-null   int64
dtypes: int64(2)
memory usage: 22.8 KB


In [55]:
# hacemos un left join de las cuentas regulatorias con la tabla anterior
  #(tabla donde agregamos subcuentas)

df_final = df3.merge(df, how='left')
df_final

,Cuenta regulatoria,Cuenta interna,Subcuentas
0,3000,1101,110101
1,3000,1101,110103
2,3000,1101,110104
3,3000,1101,110105
4,3000,1101,110106
...,...,...,...
5111,2357000,649190,6491900801
5112,2362000,570101,NaN
5113,2363000,570102,NaN
5114,2376000,6601,NaN


In [57]:
# de la tabla anterior pegamos la descripción de las cuentas regulatorias
df_final_1 = df_final.merge(df2, left_on='Cuenta regulatoria', right_on='CUENTA', how='left')
df_final_1

,Cuenta regulatoria,Cuenta interna,Subcuentas,CUENTA,Descripción
0,3000,1101,110101,3000,100200102001 Caja
1,3000,1101,110103,3000,100200102001 Caja
2,3000,1101,110104,3000,100200102001 Caja
3,3000,1101,110105,3000,100200102001 Caja
4,3000,1101,110106,3000,100200102001 Caja
...,...,...,...,...,...
5111,2357000,649190,6491900801,2357000,602001002136 Otros gastos de administración y ...
5112,2362000,570101,NaN,2362000,502209504121 Pertenecientes al sector financiero
5113,2363000,570102,NaN,2363000,502209504122 No pertenecientes al sector finan...
5114,2376000,6601,NaN,2376000,602401202144 Impuestos a la utilidad causados


In [59]:
#acomodamos el orden de las columnas como está en la hoja "Ejemplo de resultado"
df_final_2 = df_final_1[['Descripción', 'Cuenta regulatoria', 'Cuenta interna', 'Subcuentas']]
df_final_2

,Descripción,Cuenta regulatoria,Cuenta interna,Subcuentas
0,100200102001 Caja,3000,1101,110101
1,100200102001 Caja,3000,1101,110103
2,100200102001 Caja,3000,1101,110104
3,100200102001 Caja,3000,1101,110105
4,100200102001 Caja,3000,1101,110106
...,...,...,...,...
5111,602001002136 Otros gastos de administración y ...,2357000,649190,6491900801
5112,502209504121 Pertenecientes al sector financiero,2362000,570101,NaN
5113,502209504122 No pertenecientes al sector finan...,2363000,570102,NaN
5114,602401202144 Impuestos a la utilidad causados,2376000,6601,NaN


In [60]:
# guardar archivo final en formato excel

df_final_2.to_excel('archivo.xlsx', index=False)

In [61]:
#descargar el archivo
from google.colab import files

# Descargar el archivo Excel guardado
files.download('archivo.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>